In [13]:
import time, numpy as np, tensorflow as tf
from tensorflow.keras import Sequential, layers
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.callbacks import EarlyStopping

SEED = 42
tf.random.set_seed(SEED)
np.random.seed(SEED)


In [14]:
max_features = 10_000      # vocab size used in class
max_len = 500              # pad/truncate length used in class

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
x_train = sequence.pad_sequences(x_train, maxlen=max_len)
x_test  = sequence.pad_sequences(x_test,  maxlen=max_len)

print("Train:", x_train.shape, "Test:", x_test.shape)


Train: (25000, 500) Test: (25000, 500)


In [15]:
model = Sequential([
    layers.Embedding(max_features, 128, input_length=max_len),
    layers.SimpleRNN(128, activation="tanh"),   # stable choice for RNNs
    layers.Dense(1, activation="sigmoid")
])

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Force weight initialization so summary is fully populated
model.build(input_shape=(None, max_len))
model.summary()   # <-- paste this into your Word report


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ (None, 500, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_3 (SimpleRNN)        │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,313,025 (5.01 MB)

 Trainable params: 1,313,025 (5.01 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
earlystopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

start = time.time()
history = model.fit(
    x_train, y_train,
    epochs=10,
    batch_size=32,
    validation_split=0.2,
    callbacks=[earlystopping],
    verbose=1
)
elapsed = time.time() - start
print(f"Total training time: {elapsed:.2f} sec")  


Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 41s 64ms/step - accuracy: 0.6660 - loss: 0.5983 - val_accuracy: 0.7388 - val_loss: 0.5535
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 34s 53ms/step - accuracy: 0.7724 - loss: 0.4807 - val_accuracy: 0.8180 - val_loss: 0.4345
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 29s 47ms/step - accuracy: 0.7597 - loss: 0.5010 - val_accuracy: 0.5312 - val_loss: 0.7044
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 29s 47ms/step - accuracy: 0.7265 - loss: 0.5401 - val_accuracy: 0.7398 - val_loss: 0.5616
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 29s 47ms/step - accuracy: 0.7210 - loss: 0.5461 - val_accuracy: 0.6526 - val_loss: 0.6521
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 29s 47ms/step - accuracy: 0.7812 - loss: 0.4644 - val_accuracy: 0.7064 - val_loss: 0.6465
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 29s 47ms/step - accuracy: 0.8023 - loss: 0.4468 - val_accuracy: 0.6976 - val_loss: 0.7081
Total training time: 221.36 sec
